In [7]:
#Gathering Data

import requests
import pandas as pd

#chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://ejscreen.epa.gov/mapper/EJAPIinstructions.pdf
#https://ejscreen.epa.gov/mapper/ejsoefielddesc1.html
#https://www.epa.gov/ejscreen/ejscreen-api

# Load the CSV file
file_path = 'Chicago_Tract_ID.csv'
chicago_tracts_df = pd.read_csv(file_path)

def process_tract_id(tract_id):
    # Remove the dot and convert to an integer
    processed_id = int(tract_id * 100)
    # Convert to string
    processed_str = str(processed_id)
    # Pad with zeros on the right to ensure it's 6 digits long
    processed_str = processed_str.ljust(6, '0')
    return processed_str

base_fips = "17031"

# Apply the processing function to the DataFrame
chicago_tracts_df['Processed'] = chicago_tracts_df.iloc[:, 0].apply(process_tract_id)
chicago_tracts_df['FIPS'] = base_fips + chicago_tracts_df['Processed']

# Convert the list of FIPS codes to a set for efficiency
fips_codes = set(chicago_tracts_df['FIPS'])

# DataFrame to store all results
all_data = pd.DataFrame()

# Initialize counters for tracking API calls
success_count = 0
fail_count = 0

# Limit the loop to 100 API calls
for i, fips_code in enumerate(fips_codes):
    if i >= 20:
        break  # Exit the loop after n iterations

    # Modify the areatype to "tract"
    url = f"https://ejscreen.epa.gov/mapper/ejscreenRESTbroker1.aspx?namestr={fips_code}&geometry=&distance=&unit=9035&areatype=tract&areaid={fips_code}&f=json"
    response = requests.get(url)
    
    data = response.json()
    if 'data' in data:
        df = pd.json_normalize(data['data'])
        all_data = pd.concat([all_data, df], ignore_index=True)
        success_count += 1
    else:
        fail_count += 1

# Display the counts and the first few rows of the combined data
print (f"Success: {success_count}, Failures: {fail_count}")
all_data[["main.NUM_HOSPITAL","demographics.P_NHBLACK","demographics.TOTALPOP", "main.NUM_TRI"]]

pd.head(all_data)

Success: 16, Failures: 4


AttributeError: module 'pandas' has no attribute 'head'

In [5]:
#Exploring Data

all_data.head(50)
all_data.describe()
all_data.shape


NameError: name 'all_data' is not defined

In [4]:
#Visualizing Data

#If you don't already have seaborn
# %pip install seaborn
import seaborn as sns 
import pandas as pd

#Setting the theme
# sns.set(style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=True, rc=None)

#Convert to numeric
all_data[["demographics.P_NHBLACK", "main.NUM_AIRPOLL", "main.NUM_TRI"]] = all_data[["demographics.P_NHBLACK", "main.NUM_AIRPOLL", "main.NUM_TRI"]].apply(pd.to_numeric)


sns.pairplot(all_data, vars =["demographics.P_NHBLACK", "main.NUM_AIRPOLL", "main.NUM_TRI"],height=3)
# sns.scatterplot(x="demographics.P_NHBLACK", y="extras.RAW_CI_FLOOD", data= all_data)


#Outliers problematic

NameError: name 'all_data' is not defined